In [2]:
!pip install beautifulsoup4

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


make beautifulsoup object of website

In [3]:
url = r'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
import bs4
import requests

r  = requests.get(url)

data = r.text

soup = bs4.BeautifulSoup(data, 'lxml')

convert website contents to dataframe

In [4]:
import pandas as pd
my_table = soup.find('table',{'class':'wikitable sortable'})
entries = my_table.find_all('td')

cols = ['postcode', 'borough', 'neighborhood']
data_dict = {'postcode': [], 'borough': [], 'neighborhood': []}

it = iter(entries)

while True:
    for col in cols:
        entry = next(it, False)
        if not entry:
            break
        if entry.string is None:
            data_dict[col].append(entry.contents[0].attrs['title'])
        else:
            data_dict[col].append(entry.string)
    else:
        continue
    
    break

df = pd.DataFrame(data_dict)
df = df[cols]

print(df[:10])

  postcode           borough            neighborhood
0      M1A      Not assigned           Not assigned

1      M2A      Not assigned           Not assigned

2      M3A        North York               Parkwoods
3      M4A        North York        Victoria Village
4      M5A  Downtown Toronto  Harbourfront (Toronto)
5      M5A  Downtown Toronto             Regent Park
6      M6A        North York        Lawrence Heights
7      M6A        North York          Lawrence Manor
8      M7A      Queen's Park           Not assigned

9      M8A      Not assigned           Not assigned



take out the rows with a borough that is not assigned

In [5]:
df = df[df.borough != 'Not assigned']
print(df[:10])

   postcode           borough            neighborhood
2       M3A        North York               Parkwoods
3       M4A        North York        Victoria Village
4       M5A  Downtown Toronto  Harbourfront (Toronto)
5       M5A  Downtown Toronto             Regent Park
6       M6A        North York        Lawrence Heights
7       M6A        North York          Lawrence Manor
8       M7A      Queen's Park           Not assigned

10      M9A         Etobicoke        Islington Avenue
11      M1B       Scarborough          Rouge, Toronto
12      M1B       Scarborough        Malvern, Toronto


merge the neighborhoods that have the same postcode

In [6]:
def tmp(group_series):
    return pd.Series(data={'borough': group_series.iloc[0].borough, 'neighborhood': ', '.join([neighborhood.strip() for neighborhood in group_series.neighborhood])})
    
df = df.groupby('postcode').agg(tmp)

# df.reset_index(inplace=True)

print(df[:20])

              borough                                       neighborhood
postcode                                                                
M1B       Scarborough                   Rouge, Toronto, Malvern, Toronto
M1C       Scarborough  Highland Creek (Toronto), Rouge Hill, Port Uni...
M1E       Scarborough  Guildwood, Morningside, Toronto, West Hill, To...
M1G       Scarborough                                    Woburn, Toronto
M1H       Scarborough                                          Cedarbrae
M1J       Scarborough                                Scarborough Village
M1K       Scarborough  East Birchmount Park, Ionview, Kennedy Park, T...
M1L       Scarborough  Clairlea, Golden Mile, Toronto, Oakridge, Toronto
M1M       Scarborough  Cliffcrest, Cliffside, Toronto, Scarborough Vi...
M1N       Scarborough                        Birch Cliff, Cliffside West
M1P       Scarborough  Dorset Park, Scarborough Town Centre, Wexford ...
M1R       Scarborough                Maryvale, Toro

if the neighborhood is not assigned but the borough is, set the neighborhood to the borough

In [7]:
df.loc[df.neighborhood == 'Not assigned', 'neighborhood'] = df.loc[df.neighborhood == 'Not assigned', 'borough']

In [8]:
df.shape

(103, 2)

non-working code to get coordinates through geocoder

In [9]:
# !pip install geocoder

In [10]:
# import geocoder # import geocoder

# latitude_list = []
# longitude_list = []

# # initialize your variable to None

# for postcode in df.postcode:
#     lat_lng_coords = None

#     # loop until you get the coordinates
#     while(lat_lng_coords is None):
#         print('trying')
#         g = geocoder.google('{}, Toronto, Ontario'.format(postcode))
#         lat_lng_coords = g.latlng
    
#     print(postcode, lat_lng_coords)

#     latitude_list.append(lat_lng_coords[0])
#     longitude_list.append(lat_lng_coords[1])

# print(latitude_list, longitude_list)

get latitudes and longitudes from csv-file

In [11]:
df_lat_long = pd.read_csv('https://cocl.us/Geospatial_data')
df_lat_long.set_index('Postal Code', inplace=True)
print(df_lat_long.head())


              Latitude  Longitude
Postal Code                      
M1B          43.806686 -79.194353
M1C          43.784535 -79.160497
M1E          43.763573 -79.188711
M1G          43.770992 -79.216917
M1H          43.773136 -79.239476


In [12]:
df_merged = pd.concat([df, df_lat_long], axis=1)
print(df_merged.head())
print(df_merged.columns, df_merged.shape)

         borough                                       neighborhood  \
M1B  Scarborough                   Rouge, Toronto, Malvern, Toronto   
M1C  Scarborough  Highland Creek (Toronto), Rouge Hill, Port Uni...   
M1E  Scarborough  Guildwood, Morningside, Toronto, West Hill, To...   
M1G  Scarborough                                    Woburn, Toronto   
M1H  Scarborough                                          Cedarbrae   

      Latitude  Longitude  
M1B  43.806686 -79.194353  
M1C  43.784535 -79.160497  
M1E  43.763573 -79.188711  
M1G  43.770992 -79.216917  
M1H  43.773136 -79.239476  
Index(['borough', 'neighborhood', 'Latitude', 'Longitude'], dtype='object') (103, 4)


Foursquare credentials

In [84]:
# The code was removed by Watson Studio for sharing.

get nearby venues

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
toronto_venues = getNearbyVenues(names=df_merged['neighborhood'],
                                   latitudes=df_merged['Latitude'],
                                   longitudes=df_merged['Longitude']
                                  )

Rouge, Toronto, Malvern, Toronto
Highland Creek (Toronto), Rouge Hill, Port Union, Toronto
Guildwood, Morningside, Toronto, West Hill, Toronto
Woburn, Toronto
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park, Toronto
Clairlea, Golden Mile, Toronto, Oakridge, Toronto
Cliffcrest, Cliffside, Toronto, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Toronto, Wexford, Toronto
Agincourt, Toronto
Clarks Corners, Sullivan, Tam O'Shanter – Sullivan
Agincourt North, L'Amoreaux East, Milliken, Ontario, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale, Toronto
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights, Toronto
Northwood Park, York University
CFB Toronto, Downsview East
Downsview
Downsview Cen

In [18]:
print(toronto_venues.shape, toronto_venues.head())

(2246, 7)                                         Neighborhood  Neighborhood Latitude  \
0                   Rouge, Toronto, Malvern, Toronto              43.806686   
1  Highland Creek (Toronto), Rouge Hill, Port Uni...              43.784535   
2  Highland Creek (Toronto), Rouge Hill, Port Uni...              43.784535   
3  Guildwood, Morningside, Toronto, West Hill, To...              43.763573   
4  Guildwood, Morningside, Toronto, West Hill, To...              43.763573   

   Neighborhood Longitude                            Venue  Venue Latitude  \
0              -79.194353                          Wendy's       43.807448   
1              -79.160497                RIGHT WAY TO GOLF       43.785177   
2              -79.160497            Royal Canadian Legion       43.782533   
3              -79.188711  Swiss Chalet Rotisserie & Grill       43.767697   
4              -79.188711                G & G Electronics       43.765309   

   Venue Longitude        Venue Category  
0  

convert de different venues into different columns (one hot encoding)

In [31]:
# one hot encoding
df_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
col_list = list(df_onehot.columns.values)
neighborhood_index = col_list.index('Neighborhood')
fixed_columns = [col_list.pop(neighborhood_index)] + col_list
df_onehot = df_onehot[fixed_columns]

df_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Toronto, Malvern, Toronto",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek (Toronto), Rouge Hill, Port Uni...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek (Toronto), Rouge Hill, Port Uni...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, Toronto, West Hill, To...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, Toronto, West Hill, To...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


group them by neighborhood

In [42]:
df_grouped = df_onehot.groupby('Neighborhood').mean().reset_index()
df_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.01,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0
1,"Agincourt North, L'Amoreaux East, Milliken, On...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
2,"Agincourt, Toronto",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.1,0.0,0.0,0.00,0.0,0.0,0.0
4,"Alderwood, Toronto, Long Branch, Toronto",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


apply K-means clustering

In [41]:
# import k-means from clustering stage
from sklearn.cluster import KMeans


# set number of clusters
kclusters = 5

df_grouped_clustering = df_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,
       2, 0, 0, 0, 0, 0, 0], dtype=int32)

give most common venues for all neighborhoods

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [50]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = df_grouped['Neighborhood']

for ind in np.arange(df_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,American Restaurant,Thai Restaurant,Steakhouse,Cosmetics Shop,Hotel,Burger Joint,Bakery
1,"Agincourt North, L'Amoreaux East, Milliken, On...",Park,Coffee Shop,Playground,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
2,"Agincourt, Toronto",Lounge,Clothing Store,Skating Rink,Breakfast Spot,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Coffee Shop,Pizza Place,Sandwich Place,Fast Food Restaurant,Beer Store,Fried Chicken Joint,Video Store,Pharmacy,Eastern European Restaurant
4,"Alderwood, Toronto, Long Branch, Toronto",Pizza Place,Coffee Shop,Gym,Pharmacy,Skating Rink,Sandwich Place,Athletics & Sports,Pool,Pub,Yoga Studio


In [51]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
df_grouped_merged = df_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='neighborhood')

df_grouped_merged.head() # check the last columns!

,borough,neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
M1B,Scarborough,"Rouge, Toronto, Malvern, Toronto",43.806686,-79.194353,1.0,Fast Food Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Dessert Shop
M1C,Scarborough,"Highland Creek (Toronto), Rouge Hill, Port Uni...",43.784535,-79.160497,0.0,Golf Course,Bar,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
M1E,Scarborough,"Guildwood, Morningside, Toronto, West Hill, To...",43.763573,-79.188711,0.0,Medical Center,Electronics Store,Spa,Intersection,Pizza Place,Rental Car Location,Breakfast Spot,Mexican Restaurant,Dog Run,Diner
M1G,Scarborough,"Woburn, Toronto",43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Insurance Office,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Hakka Restaurant,Caribbean Restaurant,Bank,Bakery,Fried Chicken Joint,Thai Restaurant,Athletics & Sports,Coworking Space,Creperie,Falafel Restaurant


display the different neighborhoods within their clusters in a map

In [76]:
# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
pip install folium
import folium # map rendering library
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[df_merged.Latitude[0], df_merged.Longitude[0]], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# df_grouped_merged.loc[:, 'Cluster Labels'] = df_grouped_merged.loc[:, 'Cluster Labels'].astype(int)

# max_cluster = max()
# min_cluster = min(df_grouped_merged['Cluster Labels'])

# print(max_cluster, min_cluster)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_grouped_merged['Latitude'], df_grouped_merged['Longitude'], df_grouped_merged['neighborhood'], df_grouped_merged['Cluster Labels']):
    try:
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[int(cluster)-1], #cm.rainbow((cluster-min_cluster)/(max_cluster-min_cluster)), #
            fill=True,
            fill_color=rainbow[int(cluster)-1], #cm.rainbow((cluster-min_cluster)/(max_cluster-min_cluster)),
            fill_opacity=0.7).add_to(map_clusters)
    except ValueError:
        pass
       
map_clusters

give a list for all the different clusters

In [77]:
df_grouped_merged.loc[df_grouped_merged['Cluster Labels'] == 0, df_grouped_merged.columns[[1] + list(range(5, df_grouped_merged.shape[1]))]]

,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
M1C,"Highland Creek (Toronto), Rouge Hill, Port Uni...",Golf Course,Bar,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
M1E,"Guildwood, Morningside, Toronto, West Hill, To...",Medical Center,Electronics Store,Spa,Intersection,Pizza Place,Rental Car Location,Breakfast Spot,Mexican Restaurant,Dog Run,Diner
M1G,"Woburn, Toronto",Coffee Shop,Korean Restaurant,Insurance Office,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
M1H,Cedarbrae,Hakka Restaurant,Caribbean Restaurant,Bank,Bakery,Fried Chicken Joint,Thai Restaurant,Athletics & Sports,Coworking Space,Creperie,Falafel Restaurant
M1J,Scarborough Village,Playground,Convenience Store,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
M1K,"East Birchmount Park, Ionview, Kennedy Park, T...",Discount Store,Department Store,Bus Station,Hobby Shop,Coffee Shop,Convenience Store,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop
M1L,"Clairlea, Golden Mile, Toronto, Oakridge, Toronto",Bus Line,Bakery,Park,Metro Station,Fast Food Restaurant,Intersection,Soccer Field,Discount Store,Dog Run,Doner Restaurant
M1M,"Cliffcrest, Cliffside, Toronto, Scarborough Vi...",Motel,Movie Theater,American Restaurant,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
M1N,"Birch Cliff, Cliffside West",College Stadium,Skating Rink,General Entertainment,Café,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
M1P,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Thrift / Vintage Store,Pet Store,Vietnamese Restaurant,Latin American Restaurant,Chinese Restaurant,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store


In [78]:
df_grouped_merged.loc[df_grouped_merged['Cluster Labels'] == 1, df_grouped_merged.columns[[1] + list(range(5, df_grouped_merged.shape[1]))]]

,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
M1B,"Rouge, Toronto, Malvern, Toronto",Fast Food Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Dessert Shop


In [79]:
df_grouped_merged.loc[df_grouped_merged['Cluster Labels'] == 2, df_grouped_merged.columns[[1] + list(range(5, df_grouped_merged.shape[1]))]]


,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
M3K,"CFB Toronto, Downsview East",Park,Airport,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
M4W,"Rosedale, Toronto",Park,Trail,Playground,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
M8X,"The Kingsway, Montgomery Road, Old Mill North",Park,River,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
M9N,"Weston, Toronto",Park,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant


In [80]:
df_grouped_merged.loc[df_grouped_merged['Cluster Labels'] == 3, df_grouped_merged.columns[[1] + list(range(5, df_grouped_merged.shape[1]))]]

,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
M9B,"Cloverdale, Islington, Toronto, Martin Grove, ...",Bank,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Fast Food Restaurant


In [81]:
df_grouped_merged.loc[df_grouped_merged['Cluster Labels'] == 4, df_grouped_merged.columns[[1] + list(range(5, df_grouped_merged.shape[1]))]]

,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
M9M,"Emery, Toronto, Humberlea",Baseball Field,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio,Fast Food Restaurant
